# COMP9900/3900_Microservices_Project_Template

# Import libraries
Import all the related libraries

In [2]:
import panel as pn
from google.colab import files
import importlib
import os
import sys
import inspect
import ast  # For safely evaluating the input string

pn.extension(sizing_mode = 'stretch_both')
pn.extension(comms='colab')

# Upload your function file
You will need to upload your microservices file "func.py" to enable the services. \
you can download the "func.py" file from: \
https://drive.google.com/file/d/1JKtJDQY39Oxq6_AOGrU6Fw2UAFrJWK0G/view?usp=drive_link


In [3]:
# Step 1: Create the File Upload Widget
uploaded = files.upload()

# Step 2: Save the Uploaded File
uploaded_file_name = None
for filename in uploaded.keys():
    # Check if the file already exists
    if os.path.exists(filename):
        print(f"File {filename} already exists. Deleting it.")
        os.remove(filename)

    with open(filename, 'wb') as f:
        f.write(uploaded[filename])
    uploaded_file_name = filename.split('.')[0]  # Capture the module name without extension

# Step 3: Import the Functions Dynamically
if uploaded_file_name:
    # If the module was imported before, reload it to get the new changes
    if uploaded_file_name in globals():
        imported_module = importlib.reload(globals()[uploaded_file_name])
    else:
        imported_module = importlib.import_module(uploaded_file_name)


# Now you can use the functions defined in the uploaded .py file
# For example, if there's a function named 'hello' in the uploaded file:
# result = uploaded_module.hello()


Saving func.py to func.py
File func.py already exists. Deleting it.


# Import your uploaded services (lbirary)

In [4]:
import func

# Check your functions from the file

In [5]:
# Listing all the functions in the imported module
functions = [(name, obj) for name, obj in vars(func).items() if callable(obj) and not name.startswith("__")]

print("Functions in func.py:")
for name, function in functions:
    # Getting the signature of the function
    signature = inspect.signature(function)

    # Getting the docstring of the function
    doc = inspect.getdoc(function)

    print(f"Function Name: {name}")
    print(f"Signature: {name}{signature}")
    if doc:
        print(f"Docstring: {doc}")
    print("-------------------------------")


Functions in func.py:
Function Name: si
Signature: si(ticker, session=None)
-------------------------------
Function Name: import_marketdata_yahoo_csv
Signature: import_marketdata_yahoo_csv(ticker: str = 'msft', start_date: str = '2022-09-19', end_date: str = '2023-09-19', output_file_path: str = 'yfinance_ohlc.csv')
Docstring: Imports market data from Yahoo using the yfinance Ticker API.

Parameters:
- ticker (str): The stock ticker symbol. Default is 'msft'.
- start_date (str): The start date for the data in 'YYYY-MM-DD' format. Default is one year ago from today.
- end_date (str): The end date for the data in 'YYYY-MM-DD' format. Default is today.
- output_file_path (str): The path to save the resulting CSV file. Default is 'yfinance_ohlc.csv'.

Returns:
- pd.DataFrame: A dataframe containing the imported market data.
-------------------------------
Function Name: calculate_moving_average
Signature: calculate_moving_average(window: int = 20, column: str = 'Close', input_file_path: s

# Test the function

In [6]:
test1= func.import_marketdata_yahoo_csv()
test1.head()

,Open,High,Low,Close,Volume,Dividends,Stock Splits,Datetime
Date,,,,,,,,
2022-09-19 00:00:00-04:00,240.149971,242.794422,238.545476,242.180359,26826900,0.0,0.0,2022-09-19 00:00:00-04:00
2022-09-20 00:00:00-04:00,239.753804,241.180013,237.347047,240.130157,26660300,0.0,0.0,2022-09-20 00:00:00-04:00
2022-09-21 00:00:00-04:00,241.932739,245.290301,236.614111,236.663635,28625600,0.0,0.0,2022-09-21 00:00:00-04:00
2022-09-22 00:00:00-04:00,235.593952,241.526642,235.296834,238.674194,31061200,0.0,0.0,2022-09-22 00:00:00-04:00
2022-09-23 00:00:00-04:00,237.247988,238.822786,232.949518,235.643494,34176000,0.0,0.0,2022-09-23 00:00:00-04:00


# A simple interface to create data pipleline

The exmaple uses text input and Python Dictionaries to form a data pipeline. First download data from yahoo finance API, then use the downloaded data to calculate the moving average values of a specific column. You can easily change the input parameters for different results.

In [12]:
# Initialize the text area input and button
compute_button = pn.widgets.Button(name='Compute', button_type='primary',height=50)
output_text= pn.widgets.TextAreaInput(value='',placeholder= 'Results will be shown here')
text_input = pn.widgets.TextAreaInput(value='''{
              "import_marketdata_yahoo_csv": {
                                'ticker' : 'msft',
                                'start_date'  : '2022-04-01',
                                'end_date'  : '2023-03-31',
                                'output_file_path'  : 'yfinance_ohlc.csv' },

              "calculate_moving_average": {
                  'window': 20,
                   'column': 'Close',
                  'input_file_path': 'yfinance_ohlc.csv',
                  'output_file_path': 'yfinance_ohlc_ma2.csv'},
          }''', height=500)

# Define the callback function
def compute_callback(event):
    output_text.value = ''
    # Get the text from the text_input
    text = text_input.value
    # You can now do something with the text, like processing it or displaying results
    # For this example, let's just print it (or display it somewhere in your panel app)
    pipeline_dict = ast.literal_eval(text)
    for function_name, parameters in pipeline_dict.items():
        module = 'func.'
        function = eval(module+function_name)
        result = function(**parameters)

        output_text.value += "\n===================="+function_name+"====================\n\n"
        output_text.value += str(result)


# Link the button click event to the callback function
compute_button.on_click(compute_callback)

# Create a panel layout and show it
app =pn.Row( pn.Column(text_input,compute_button) , output_text)

app

Row(sizing_mode='stretch_both')
    [0] Column(sizing_mode='stretch_both')
        [0] TextAreaInput(height=500, sizing_mode='stretch_width', value='{\n              "import_...)
        [1] Button(button_type='primary', height=50, name='Compute', sizing_mode='stretch_width')
    [1] TextAreaInput(placeholder='Results will b..., sizing_mode='stretch_both')